# Objective and Motivation

## Objective

## Overview of Wind Turbine Anatomy

## Wind Turbine Aerodynamics

# Code

## Prepare Data

### Pip Install, Init, and Start Pyspark

In [ ]:
# from pyspark.sql import SparkSession
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
# from pyspark.sql import functions as F
# from pyspark.sql import types as T
# from pyspark.sql.functions import broadcast
# from pyspark.sql.functions import col
# from pyspark.sql.functions import *
import seaborn as sns

%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, f1_score, mean_absolute_percentage_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.tools import diff
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
import mlflow
import optuna

In [ ]:
Path.cwd().joinpath("mlruns").as_uri()

In [ ]:
# from ydata_profiling import ProfileReport

In [ ]:
# spark = SparkSession.builder.master("local[*]").getOrCreate()
# sc = spark.sparkContext

In [ ]:
# Get the number of cores
# num_cores = sc.defaultParallelism

# Get the number of executors
# num_executors = sc.getConf().get("spark.executor.instances")

# print(num_cores)
# print(num_executors)

### Import Data 

In [ ]:
print(os.getcwd())

In [ ]:
df = pd.read_csv("../src/main/resources/data/Turbine_Data.csv")

In [ ]:
# df = (spark.read
#       .option("header", "true")
#       .option("inferSchema", "true")
#       .csv("../data/Turbine_Data.csv"))

In [ ]:
# df.printSchema()

### Transform Data

In [ ]:
# df_renamed = df.withColumnRenamed("_c0", "datetime_stamp")


In [ ]:
df_renamed = df.rename({"Unnamed: 0": "datetime_stamp"}, axis=1)

In [ ]:
df_renamed.columns

In [ ]:
df_renamed['datetime_stamp'] = pd.to_datetime(df_renamed['datetime_stamp'])

In [ ]:
df_indexed_sorted = df_renamed.set_index("datetime_stamp").sort_index()

In [ ]:
df_indexed_sorted.dtypes

In [ ]:
pdf = df_indexed_sorted.copy()

In [ ]:
# df_casted = df_renamed.withColumn("datetime_stamp", col("datetime_stamp").cast("timestamp")).drop(col("_c0"))
# df_casted.printSchema()

In [ ]:
# df_filtered = df_casted.where(
#     to_date(col("datetime_stamp")) =="2018-01-01"
#     )

In [ ]:
# pdf = df_casted.toPandas()

In [ ]:
# profile = ProfileReport(pdf, title="YData Profiling Report")
# profile.to_file('profiling_report.html');

### Drop Empty or Single Value Cols

In [ ]:
pdf_drop_bad_cols = pdf.drop(columns=['ControlBoxTemperature', 'TurbineStatus', 'WTG'], axis=1)

In [ ]:
type(pdf_drop_bad_cols)

### Drop Highly Correlated Columns

In [ ]:
corr_matrix = pdf_drop_bad_cols.corr()
corr_matrix


In [ ]:
corr_matrix.abs().unstack()["NacellePosition"]

In [ ]:
corr_matrix.abs().unstack().sort_values(ascending=False).drop_duplicates().head(20)

In [ ]:
### Drop highly correlated features
### pdf[["WindSpeed","ActivePower"]]
### pdf[["RotorRPM", "GeneratorRPM"]]
### pdf[["AmbientTemperatue", "MainBoxTemperature"]]
### pdf[["Blade1PitchAngle","Blade2PitchAngle", "Blade3PitchAngle"]]
### pdf[["NacellePosition","WindDirection"]]


df_drop_corr_cols = pdf.drop(["NacellePosition","RotorRPM", "AmbientTemperatue", "Blade2PitchAngle", "Blade3PitchAngle"], axis=1)


In [ ]:
df_drop_corr_cols

### Drop Null Rows

In [ ]:
df_drop_nulls = df_drop_corr_cols.dropna(subset=["ActivePower"]).sort_index()
# df_drop_nulls = df_drop_corr_cols.sort_index()

In [ ]:
### Percent non-null
df_drop_nulls.shape[0]/pdf.shape[0]

In [ ]:
df_drop_nulls.shape

### Train/Test Split

In [ ]:
def train_test_split_by_date(df: pd.DataFrame, train_test_date_split_mid_pt: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    X_train = df.iloc[df.index <= train_test_date_split_mid_pt]
    X_test = df.iloc[df.index > train_test_date_split_mid_pt]
    # print(X_train)
    # print(X_test)
    return X_train, X_test

train_test_date_split_mid_pt = "2019-03-01"

X_train_time_series, X_test_time_series = train_test_split_by_date(df_drop_nulls, train_test_date_split_mid_pt)

In [ ]:
### Plot x_train and x_test
y_var = "ActivePower"

plt.figure(figsize=(10, 6))
plt.plot(X_train_time_series[y_var])
plt.plot(X_test_time_series[y_var])
plt.title("Active Power Generated")
plt.xlabel("Date")
plt.ylabel("ActivePower")
time_series_fn = f"./figures/time_series.png"
plt.savefig(time_series_fn)
plt.close()
# plt.show()

In [ ]:
def resample_mean(X_train: pd.DataFrame, X_test: pd.DataFrame, y_variable: str, grain: str) -> tuple[pd.Series, pd.Series]:
    return X_train[y_variable].resample(grain).mean(), X_test[y_variable].resample(grain).mean()

X_train_resampled, X_test_resampled = resample_mean(X_train_time_series, X_test_time_series, y_variable=y_var, grain="W-MON")

In [ ]:
### Plot x_train and x_test
plt.figure(figsize=(10, 6))
plt.plot(X_train_resampled)
plt.plot(X_test_resampled)
plt.title("Active Power Generated")
plt.xlabel("Date")
plt.ylabel("ActivePower")
time_series_resampled_fn = f"./figures/time_series_resampled.png"
plt.savefig(time_series_resampled_fn)
plt.close()
# plt.show()

In [ ]:
# def _apply_differencing(self, time_series: pd.Series):
    # return diff(time_series, k_seasonal_diff=1, seasonal_periods=self.seasonal_periods)


## Fit and Forecast (SARIMA)

In [ ]:
import mlflow
import mlflow.data

mlflow.set_experiment("wind-energy-forecasting")

# Enable autologging for scikit-learn
mlflow.statsmodels.autolog()

### Evaluation Functions

In [ ]:
def check_stationarity(self, time_series: pd.Series) -> None:
    result = adfuller(time_series, autolag='AIC')
    p_value = result[1]
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {p_value}')
    print('Stationary' if p_value < 0.05 else 'Non-Stationary')
    return None

In [ ]:
def evaluate_forecast(X_test: pd.Series, forecast) -> tuple[float, float, float, float]:
    # print(X_test[1:].sort_index())
    # print(forecast.sort_index())

    mae = mean_absolute_error(X_test[1:], forecast)
    mape = mean_absolute_percentage_error(X_test[1:], forecast)
    mse = mean_squared_error(X_test[1:], forecast)
    rmse = root_mean_squared_error(X_test[1:], forecast)

    print(f'MAE: {mae}')
    print(f'MAE: {mape}')
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')

    return mae, mape, mse, rmse

### Optuna

In [ ]:
def objective(trial):
    # Setting nested=True will create a child run under the parent run.
    with mlflow.start_run(nested=True, run_name=f"trial_{trial.number}") as child_run:
        max_p = trial.suggest_int("max_p", 1, 3)
        max_d = trial.suggest_int("max_d", 1, 3)
        max_q = trial.suggest_int("max_q", 1, 3)
        max_P = trial.suggest_int("max_P", 1, 3)
        max_D = trial.suggest_int("max_D", 1, 3)
        max_Q = trial.suggest_int("max_Q", 1, 3)
        seasonal_periods = 26
        forecast_n_steps = len(X_test_resampled) - 1

        X_train_dataset = mlflow.data.from_pandas(X_train_time_series)
        X_test_dataset = mlflow.data.from_pandas(X_test_time_series)
        X_train_resampled_dataset = mlflow.data.from_pandas(pd.DataFrame(X_train_resampled))
        X_test_resampled_dataset = mlflow.data.from_pandas(pd.DataFrame(X_test_time_series))

        params = {
            "max_p": max_p,
            "max_d": max_d,
            "max_q": max_q,
            "max_P": max_P,
            "max_D": max_D,
            "max_Q": max_Q,
            "seasonal_periods": seasonal_periods,
            "random_state": 42,
            
        }

        # Log current trial's parameters
        mlflow.log_params(params)

        # Optional: Set a tag that we can use to remind ourselves what this run was for
        mlflow.set_tag("Training Info", "Time Series forecasting via SARIMA")

        mlflow.log_inputs(
            datasets=[X_train_dataset, X_test_dataset, X_train_resampled_dataset, X_test_resampled_dataset],
        contexts=["Training", "Testing", "Training Resampled", "Testing Resampled"],
                tags_list=[None, {"my_tag": "tag_value"}, None, None],
        )


        ### Log ACF
        plot_acf(X_train_resampled)
        acf_fn = f"./figures/acf_{trial.number}.png"
        plt.savefig(acf_fn)
        mlflow.log_artifact(acf_fn) # logging to mlflow      
        plt.close()
        # plt.show()
        
        
        ### Log PACF
        plot_pacf(X_train_resampled)
        pacf_fn = f"./figures/pacf_{trial.number}.png"
        plt.savefig(pacf_fn)
        mlflow.log_artifact(pacf_fn) # logging to mlflow      
        plt.close()
        # plt.show()
                
        ## Fit and forecast
        model = SARIMAX(X_train_resampled, order=(max_p, max_d, max_q), seasonal_order=(max_P, max_D, max_Q, seasonal_periods))
        fit_model = model.fit()
        forecast = fit_model.get_forecast(steps=forecast_n_steps)
        forecast_values = forecast.predicted_mean
        forecast_ci = forecast.conf_int()
        

        ### Plot Training, Testing, Forecast, and CI
        plt.figure(figsize=(10, 6))
        plt.plot(X_train_resampled, label='Training')
        plt.plot(X_test_resampled, label='Actuals', color='orange')
        plt.plot(forecast_values, label='Forecast', color='red')
        plt.fill_between(forecast_ci.index,
                        forecast_ci.iloc[:,0],
                        forecast_ci.iloc[:,1],
                        color='k', alpha=.15)
        plt.title("Active Power Forecast")
        plt.xlabel("Date")
        plt.ylabel("Sales")
        plt.legend()
        train_test_fcst_ci_fn = f"./train_test_fcst_ci_{trial.number}.png"
        plt.savefig(train_test_fcst_ci_fn)
        mlflow.log_artifact(train_test_fcst_ci_fn) # logging to mlflow      
        plt.close
        # plt.show()

        # ## Log plot_diagnostics()
        # fit_model.plot_diagnostics()
        # diagnostics_fn = f"./figures/diagnostics_{trial.number}.png"
        # plt.savefig(diagnostics_fn)
        # mlflow.log_artifact(diagnostics_fn) # logging to mlflow      
        # plt.close()
        # # plt.show()
        
        # Predict on the test set, compute and log the loss metric
        mae, mape, mse, rmse = evaluate_forecast(X_test_resampled, forecast_values)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("mape", mape)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        # mlflow.log_metric("residuals", residuals)
        mlflow.log_metrics({"mae": mae, "mape": mape, "mse": mse, "rmse": rmse})

        # Log the model file
        # mlflow.statsmodels.log_model(fit_model, name="model", remove_data=True)

        # Make it easy to retrieve the best-performing child run later
        trial.set_user_attr("run_id", child_run.info.run_id)
        return fit_model.aic
    


In [ ]:
# Start an MLflow run)
with mlflow.start_run(run_name='SARIMAX') as run:
    n_trials = 30
    mlflow.log_param("n_trials", n_trials)

    study = optuna.create_study(direction="minimize")
    # study.optimize(objective, callbacks=[optuna.study.MaxTrialsCallback(n_trials, states=(optuna.trial.TrialState.COMPLETE,))])
    study.optimize(objective, n_trials=n_trials)

    # Log the best trial and its run ID
    mlflow.log_params(study.best_trial.params)
    mlflow.log_metrics({"best_error": study.best_value})
    if best_run_id := study.best_trial.user_attrs.get("run_id"):
        mlflow.log_param("best_child_run_id", best_run_id)



In [ ]:
### TODO: add pacf and acf for resampled
### TODO: Add next forecasating

## Sources

https://www.geeksforgeeks.org/machine-learning/sarima-seasonal-autoregressive-integrated-moving-average/

In [ ]:
### Apply Differencecing
# print("Apply Differencing")
# X_train_resampled_differenced = pd.Series(self._apply_differencing(X_train_resampled))
# X_test_resampled_differenced = pd.Series(self._apply_differencing(X_test_resampled))

### Plot X_train with differencing and resampling
# print("Plotting X_train with Differencing and resampling")
# weekly_forecast._plot_time_series(X_train_resampled, X_test_resampled)
# weekly_forecast._check_stationarity(X_train_resampled)
